# Labeling For All Content Types

Use this notebook the final index file and label Alignment Sheets and Figures based on their title. This is the scenario where we have already labeled the tables. This notebook will be removed during the code review and cleaning process.

# Imports

In [1]:
import pickle
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.util import ngrams
from pathlib import Path
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
ROOT_PATH = Path('.').resolve().parents[1]
print(ROOT_PATH)

C:\Users\t1nipun\Documents\esa-data-bank_banque-donnees-ees


# Setup

## Load our list of keywords created in keywords.py

In [3]:
with open(str(ROOT_PATH) + "\\data\\processed\\keywords_pickle\\vc_keywords.pkl", "rb") as f:
    keywords = pickle.load(f)        

In [4]:
print(keywords[14])

['navig navig safeti', 'navig', 'nonnavig', 'navig protect activ', 'guid line', 'guid wire', 'signag', 'waterway', 'cross', 'cross plan', 'impact navig', 'dewat navig water', 'navig water', 'navig waterway', 'navig', 'navig', 'watercours', 'watercours cross', 'water cross', 'cross methodolog', 'horizont direct drill', 'hdd', 'bridg', 'marin', 'marin termin', 'waterway user', 'recreat waterway user', 'navig use', 'tributari', 'tourism', 'tourist', 'guid outfitt', 'outfitt', 'angler', 'cano', 'kayak', 'boat', 'sail', 'sail', 'fisheri ocean canada', 'navig protect act', 'navig watercours', 'navig safeti', 'schedul water', 'non-schedul water', 'recreation-rel navig', 'commercial-rel navig', 'watercours user', 'waterway user', 'watercours cross', 'cross method', 'trenchless cross', 'trench cross', 'navig hazard', 'expos instream', 'buoyanc issu', 'upstream', 'downstream', 'warn sign', 'warn signag', 'instream', 'temporari vehicl cross', 'bed', 'bank', 'preconstruct contour', 'hydraul charac

In [5]:
# list of VCs

labels_list = ['Landscape, terrain, and weather', 
                'Soil',
                'Plants',
                'Water',
                'Fish',
                'Wetlands',
                'Wildlife',
                'Species at Risk',
                'Greenhouse gas emissions',
                'Air emissions',
                'Noise',
                'Electricity and electromagnetism',
                'Proximity to people',
                'Archaeological, paleontological, historical, and culturally significant sites and resources',
                'Human access to boats and waterways',
                'Indigenous land, water, and air use',
                'Impact to social and cultural well-being',
                'Impact to human health and viewscapes',
                'Social, cultural, economic infrastructure and services',
                'Economic Offsets and Impact',
                'Environmental Obligations',
                'Treaty and Indigenous Rights']

# Load Final Index File

This is where we load the 'final' index file so that we can label the figures and alignment sheets.

In [6]:
df = pd.read_csv(str(ROOT_PATH) + '\\data\\interim\\Intermediate_Index_Files\\esa_index_with_table_text_no_labels.csv', encoding='utf-8-sig')
df_fig_align = df[df['Content Type'].isin(['Figure', 'Alignment Sheet'])]

In [7]:
print(len(df), len(df_fig_align))

1032 363


In [8]:
df_fig_align.head()

,Application Filing Date,Application ID,Application Name,Application Short Name,Application Type,Application URL,Commodity,Company Name,Consultant Name,Content Type,Data ID,Decision URL,Document Number,ESA Folder URL,ESA Section(s),File Name,Hearing Order,PDF Download URL,PDF Page Count,PDF Page Number,Pipeline Location,Pipeline Status,Regulatory Instrument(s),Title,Topics,csvFileName,figureNumber,tableNumber,text,label
669,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,NaN,-1
670,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,13.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 2.1 Vegetation and Wetlands Regional Assessment Area,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,NaN,-1
671,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,5.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,NaN,-1
672,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,NaN,-1
673,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,49.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 6.1 Vegetation and Wetlands Regional Assessment Area,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,NaN,-1


## Labeling For All Content Types

Use this notebook the final index file and label Alignment Sheets and Figures based on their title. This is the scenario where we have already labeled the tables. This notebook will be removed during the code review and cleaning process.

In [9]:
df_fig_align['text'] = df_fig_align['Title']

In [10]:
df_fig_align.head()

,Application Filing Date,Application ID,Application Name,Application Short Name,Application Type,Application URL,Commodity,Company Name,Consultant Name,Content Type,Data ID,Decision URL,Document Number,ESA Folder URL,ESA Section(s),File Name,Hearing Order,PDF Download URL,PDF Page Count,PDF Page Number,Pipeline Location,Pipeline Status,Regulatory Instrument(s),Title,Topics,csvFileName,figureNumber,tableNumber,text,label
669,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,(Alignment Sheet) Figure 1.1 Project Overview,-1
670,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,13.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 2.1 Vegetation and Wetlands Regional Assessment Area,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,(Alignment Sheet) Figure 2.1 Vegetation and Wetlands Regional Assessment Area,-1
671,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,5.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,-1
672,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,-1
673,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,49.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 6.1 Vegetation and Wetlands Regional Assessment Area,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,(Alignment Sheet) Figure 6.1 Vegetation and Wetlands Regional Assessment Area,-1


In [11]:
from bs4 import BeautifulSoup

df_fig_align['text'] = df_fig_align['text'].apply(lambda x: BeautifulSoup(str(x), "html.parser").get_text(separator=' ')) # remove html tags
df_fig_align['text'] = df_fig_align['text'].replace('[^a-zA-Z0-9 ]', ' ', regex=True) # remove all non-alpha-numeric characters
df_fig_align['text'] = df_fig_align['text'].replace('\w{25,}', ' ', regex=True)
df_fig_align['text'] = df_fig_align['text'].replace('cid\d+', ' ', regex=True)
df_fig_align['text'] = df_fig_align['text'].replace(' s ', ' ', regex=True)
df_fig_align['text'] = df_fig_align['text'].replace(' +', ' ', regex=True) # remove all extra spaces in text
df_fig_align['text'] = df_fig_align['text'].str.lower()

In [12]:
df_fig_align['text'] = df_fig_align['text'].apply(lambda x: x[:30_000]) # making sure the text is not longer than 30k characters
df_fig_align.head()

,Application Filing Date,Application ID,Application Name,Application Short Name,Application Type,Application URL,Commodity,Company Name,Consultant Name,Content Type,Data ID,Decision URL,Document Number,ESA Folder URL,ESA Section(s),File Name,Hearing Order,PDF Download URL,PDF Page Count,PDF Page Number,Pipeline Location,Pipeline Status,Regulatory Instrument(s),Title,Topics,csvFileName,figureNumber,tableNumber,text,label
669,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 1 1 project overview,-1
670,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,13.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 2.1 Vegetation and Wetlands Regional Assessment Area,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 2 1 vegetation and wetlands regional assessment area,-1
671,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,5.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 1 1 project overview map,-1
672,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 1 1 project overview map,-1
673,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,49.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 6.1 Vegetation and Wetlands Regional Assessment Area,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 6 1 vegetation and wetlands regional assessment area,-1


In [13]:
table_texts = df_fig_align['text'].tolist()

tokenized_table_texts = []
stemmer = PorterStemmer()

for i, table_text in enumerate(table_texts):
    processed_text = word_tokenize(table_text)
    processed_text = [stemmer.stem(w) for w in processed_text if w not in stopwords.words("english")]
    table_ngram_list = []
    for n in range(1, 7):
        table_ngrams = list(ngrams(processed_text, n))
        table_ngram_list.extend([" ".join(table_gram) for table_gram in table_ngrams])

    tokenized_table_texts.append(table_ngram_list)
    
print(tokenized_table_texts[0][0:200])

['align', 'sheet', 'figur', '1', '1', 'project', 'overview', 'align sheet', 'sheet figur', 'figur 1', '1 1', '1 project', 'project overview', 'align sheet figur', 'sheet figur 1', 'figur 1 1', '1 1 project', '1 project overview', 'align sheet figur 1', 'sheet figur 1 1', 'figur 1 1 project', '1 1 project overview', 'align sheet figur 1 1', 'sheet figur 1 1 project', 'figur 1 1 project overview', 'align sheet figur 1 1 project', 'sheet figur 1 1 project overview']


In [14]:
token_figTxt_pkl_path = str(ROOT_PATH / "data" / "processed" / "keywords_pickle" / "tokenized_figure_alignment_sheets_texts.pkl")
with open(token_figTxt_pkl_path, 'wb') as f:
    pickle.dump(tokenized_table_texts, f)

In [15]:
with open(str(ROOT_PATH) + "\\data\\processed\\keywords_pickle\\tokenized_figure_alignment_sheets_texts.pkl", 'rb') as f:
    tokenized_table_text = pickle.load(f)

In [16]:
print(tokenized_table_text[0][:10], '\n', tokenized_table_text[-1][:50])

['align', 'sheet', 'figur', '1', '1', 'project', 'overview', 'align sheet', 'sheet figur', 'figur 1'] 
 ['align', 'sheet', '4166837', '118', 'align sheet', 'sheet 4166837', '4166837 118', 'align sheet 4166837', 'sheet 4166837 118', 'align sheet 4166837 118']


In [17]:
for label in labels_list:
    df_fig_align[f'{label}'] = 0
    # df_f_as[f'{label} - Number of Matches'] = 0
    # df_f_as[f'{label} Relevance'] = 0

df_fig_align.head(2)

,Application Filing Date,Application ID,Application Name,Application Short Name,Application Type,Application URL,Commodity,Company Name,Consultant Name,Content Type,Data ID,Decision URL,Document Number,ESA Folder URL,ESA Section(s),File Name,Hearing Order,PDF Download URL,PDF Page Count,PDF Page Number,Pipeline Location,Pipeline Status,Regulatory Instrument(s),Title,Topics,csvFileName,figureNumber,tableNumber,text,label,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights
669,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 1 1 project overview,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
670,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,13.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 2.1 Vegetation and Wetlands Regional Assessment Area,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 2 1 vegetation and wetlands regional assessment area,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
def label_with_keywords(i, table_text, keywords_for_label, label):
    number_of_matches = len([word for word in keywords_for_label if word in table_text])
    df_fig_align[f'{label}'][i] = number_of_matches

In [19]:
def string_total_sum_match_2_lists(list_1, list_2):
    """
    This function takes two lists of strings and counts the number of total matches between the two lists, including duplicates.
    The function returns the number of total matches.
    """
    # Initialize the count variable.
    count = 0
    # Iterate over the first list.
    for item in list_1:
        # If the item is in the second list, increment the count variable.
        if item in list_2:
            count += 1
    # Return the count variable.
    return count

In [20]:
df_fig_align.text.iloc[-1]

'alignment sheet 4166837 118'

In [21]:
df_fig_align.index[0]

669

In [22]:
for i, table_text in zip(df_fig_align.index, tokenized_table_text):
    for keywords_for_label, label in zip(keywords, labels_list):
        number_of_matches = string_total_sum_match_2_lists(table_text, keywords_for_label)
        df_fig_align[f'{label}'][i] = number_of_matches

esa_fig_alignment_labeled_path = str(ROOT_PATH / "data" / "processed" / "keywords_pickle" / "esa_index_ENG_fig_alignment_labeled.pkl")
with open(esa_fig_alignment_labeled_path, 'wb') as f:
    pickle.dump(df_fig_align, f)

df_fig_align.head()

,Application Filing Date,Application ID,Application Name,Application Short Name,Application Type,Application URL,Commodity,Company Name,Consultant Name,Content Type,Data ID,Decision URL,Document Number,ESA Folder URL,ESA Section(s),File Name,Hearing Order,PDF Download URL,PDF Page Count,PDF Page Number,Pipeline Location,Pipeline Status,Regulatory Instrument(s),Title,Topics,csvFileName,figureNumber,tableNumber,text,label,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights
669,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 1 1 project overview,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
670,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,13.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 2.1 Vegetation and Wetlands Regional Assessment Area,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 2 1 vegetation and wetlands regional assessment area,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
671,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,5.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 1 1 project overview map,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
672,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Alignment Sheet,4166302,NaN,A7Y7G5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,APPENDIX D VEGETATION AND WETLANDS TECHNICAL DATA REPORT,C16186-14,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166302,118,11.0,"British Columbia, Alberta",Applied,NaN,(Alignment Sheet) Figure 1.1 Project Overview Map,"['Land', 'Water', 'Vegetation', 'Technology']",NaN,1.0,NaN,alignment sheet figure 1 1 project overview map,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
673,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Mi

# Drop text and remerge index

In [23]:
df_fig_align.drop(columns=['text'], inplace=True)
df_fig_align.drop(columns=['label'], inplace=True)

In [24]:
df_fig_align.to_csv(str(ROOT_PATH) + '\\data\\interim\\Intermediate_Index_Files\\esa_figure_alignment_vec_labeled.csv', index=False, encoding='utf-8-sig')

In [25]:
df_tables = pd.read_csv(str(ROOT_PATH) + '\\data\\interim\\Intermediate_Index_Files\\esa_tables_vec_labeled.csv', encoding='utf-8-sig')

In [26]:
df_esa_vecs = df_fig_align.append(df_tables, ignore_index=True)

In [27]:
print(df_fig_align.shape, df_tables.shape, df_esa_vecs.shape)

(363, 50) (669, 50) (1032, 50)


In [38]:
df_esa_vecs.drop(columns=['ID'], inplace=True)
df_esa_vecs.insert(len(df_esa_vecs.columns), 'ID', range(21425, 21425 + len(df_esa_vecs)))

In [35]:
df_esa_vecs.to_csv(str(ROOT_PATH) + '\\data\\interim\\Intermediate_Index_Files\\esa_vecs_labeled.csv', index=False, encoding='utf-8-sig')

# Normalize match values

In [ ]:
import numpy as np
max_value_list = []
df_norm = df.copy()
for content_type in ['Table', 'Figure', 'Alignment Sheet']:
    df_content_type = df[df['Content Type'] == content_type]
    for label in labels_list:
        df_log = np.log2(df_content_type[f'{label}'].replace(0, np.nan) + 1)
        max_vc = df_log.max()
        updated_vc_col = df_log / max_vc * 100
        updated_vc_col.replace(np.nan, 0, inplace=True)
        df_content_type[f'{label}'] = np.ceil(updated_vc_col).astype(int)
        max_value_list.append([content_type, label, max_vc])
    df_norm.loc[df_content_type.index] = df_content_type

In [ ]:
(unique, counts) = np.unique(df_norm, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

In [ ]:
df_norm_tables = df_norm[df_norm['Content Type'] == 'Table']
df_norm_alignment_sheets = df_norm[df_norm['Content Type'] == 'Alignment Sheet']
df_norm_figures = df_norm[df_norm['Content Type'] == 'Figure']

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 3, sharex='col', sharey='row')

num = 1

df_norm_tables.hist(column = labels_list[num], bins = 30, ax=ax[0], figsize=(10, 10))
df_norm_alignment_sheets.hist(column = labels_list[num], bins = 30, ax=ax[1], figsize=(10, 10))
df_norm_figures.hist(column = labels_list[num], bins = 30, ax=ax[2], figsize=(10, 10))


In [ ]:
df_norm.tail()

In [ ]:
df = df_norm.copy()

# Adding Alignment Sheet in Alignment Sheet rows with Figure in Title

In [ ]:
df.loc[(df['Content Type'] == 'Alignment Sheet') & (df['Title'].str.contains('Figure')), 'Title'] = '(Alignment Sheet) ' + df.loc[(df['Content Type'] == 'Alignment Sheet'), 'Title']
df.loc[(df['Content Type'] == 'Alignment Sheet') & (df['Title'].str.contains('FIGURE')), 'Title'] = '(Alignment Sheet) ' + df.loc[(df['Content Type'] == 'Alignment Sheet'), 'Title']
df.loc[(df['Content Type'] == 'Alignment Sheet') & (df['Title'].str.contains('figure')), 'Title'] = '(Alignment Sheet) ' + df.loc[(df['Content Type'] == 'Alignment Sheet'), 'Title']

df.loc[(df['Content Type'] == 'Alignment Sheet') & (df['Title'].str.contains('Table')), 'Title'] = '(Alignment Sheet) ' + df.loc[(df['Content Type'] == 'Alignment Sheet'), 'Title']
df.loc[(df['Content Type'] == 'Alignment Sheet') & (df['Title'].str.contains('TABLE')), 'Title'] = '(Alignment Sheet) ' + df.loc[(df['Content Type'] == 'Alignment Sheet'), 'Title']
df.loc[(df['Content Type'] == 'Alignment Sheet') & (df['Title'].str.contains('table')), 'Title'] = '(Alignment Sheet) ' + df.loc[(df['Content Type'] == 'Alignment Sheet'), 'Title']
df.tail()

# Load Alignment Sheet Titles for French Index

In [ ]:
Data_Files_Path = str(Path().resolve().parents[1]) + '\\Data_Files\\'
print(Data_Files_Path)

In [ ]:
df_new_align = pd.read_csv(Data_Files_Path + "Intermediate_Index_Files\\new_alignment_sheet_titles_for_translation_FR_fixed.csv")

In [ ]:
df_new_align.tail()

# Add VCs to French Index

In [ ]:
df_fra = pd.read_csv(final_index_path + 'ESA_website_FRA.csv')

df_fra.head(2)

In [ ]:
# grabbing the VC columns to put match numbers in French file
df_fra.iloc[:, 29:51] = df.iloc[:, 29:51]
df_fra.iloc[:, 29:51].head()

In [ ]:
ids_to_update = df[df["ID Internal"].isin(df_new_align["ID"] + '_a_1')]['ID Internal'].tolist()

In [ ]:
idx_list = []
for i, id in enumerate(ids_to_update):
    id = id
    idx = df_fra[df_fra["ID Internal"] == id].index[0]
    # print(idx)
    idx_list.append(idx)
    df_fra.loc[idx, 'Titre'] = df_new_align['Titre'][i]
    # print(df_new_align['Titre'][i])
df_fra.iloc[idx_list[20:25]]

# Update the Alignment Sheet French Titles

In [ ]:
df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé') & (df_fra['Titre'].str.contains('Figure')), 'Titre'] = '(Carte-tracé) ' + df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé'), 'Titre']
df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé') & (df_fra['Titre'].str.contains('FIGURE')), 'Titre'] = '(Carte-tracé) ' + df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé'), 'Titre']
df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé') & (df_fra['Titre'].str.contains('figure')), 'Titre'] = '(Carte-tracé) ' + df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé'), 'Titre']

df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé') & (df_fra['Titre'].str.contains('Tableau')), 'Titre'] = '(Carte-tracé) ' + str(df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé'), 'Titre'])
df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé') & (df_fra['Titre'].str.contains('TABLEAU')), 'Titre'] = '(Carte-tracé) ' + str(df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé'), 'Titre'])
df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé') & (df_fra['Titre'].str.contains('tableau')), 'Titre'] = '(Carte-tracé) ' + str(df_fra.loc[(df_fra['Type de contenu'] == 'Carte-tracé'), 'Titre'])
df_fra.tail()

# 

In [ ]:
df_fra.head()

# Remove Duplicates

In [ ]:
remove_duplicate_as = ['1059614_45_a_1',
'2392795_110_a_1',
'2393296_31_a_1',
'3334565_29_a_1',
'3334565_31_a_1',
'3334565_43_a_1',
'3334565_44_a_1',
'3337530_34_a_1',
'3337530_35_a_1',
'3337530_6_a_1',
'3340309_17_a_1',
'3340309_21_a_1',
'3341938_15_a_1',
'3341938_25_a_1',
'3341938_31_a_1',
'3341938_32_a_1',
'3341938_33_a_1',
'3342531_19_a_1',
'3342531_4_a_1',
'3342531_45_a_1',
'3891804_134_a_1',
'464812_45_a_1',
'464812_46_a_1',
'464812_47_a_1',
'464812_48_a_1']

In [ ]:
for id in remove_duplicate_as:
    df.drop(df[df['ID Internal'] == id].index[0], inplace=True)
    df_fra.drop(df_fra[df_fra['ID Internal'] == id].index[0], inplace=True)

In [ ]:
df[df["ID Internal"] == remove_duplicate_as[3]]

# Sanity Checks

Let's have a look at all the rows where there were no matches for any of the VCs. We want to make sure this is only because none of the text matched with any of the keywords in our list.

In [ ]:
# After running this cell, open up an index file with the text content and search for the table title to see if it makes sense that such a row has 0 matches.

df[df.iloc[:, 29:51].sum(axis=1) == 0].head()

# Save Labeled Index File

In [ ]:
import time
current_time = time.strftime('%Y-%m-%d_%H-%M-%S')
print(current_time)

In [ ]:
save_index_path = str(Path().resolve().parents[1]) + '\\Output_Files\\final_index_files\\'

In [ ]:
df.to_csv(save_index_path + f'ESA_website_ENG_{current_time}.csv', encoding='utf-8-sig')
df_fra.to_csv(save_index_path + f'ESA_website_FRA_{current_time}.csv', encoding='utf-8-sig')